In [317]:
import numpy as np
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


In [318]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [319]:
# For subject 1
X_train_valid_1 = X_train_valid[np.where(person_train_valid==0)[0]]
y_train_valid_1 = y_train_valid[np.where(person_train_valid==0)[0]] - 769
X_test_1 = X_test[np.where(person_test==0)[0]]
y_test_1 = y_test[np.where(person_test==0)[0]] - 769
print(y_test_1)

[1 0 2 3 3 2 2 3 3 0 2 3 2 0 1 1 3 3 2 0 2 1 0 1 3 2 0 1 1 3 2 2 1 1 2 0 0
 0 0 3 1 2 0 1 3 2 0 3 2 1]


In [320]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = torch.FloatTensor(X)
        self.Y = torch.LongTensor(Y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]

In [321]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(X_train_valid_1, y_train_valid_1,
                                                              test_size=0.2,shuffle=True)


X_train_1 = X_train_1.reshape(X_train_1.shape[0],1,X_train_1.shape[1],X_train_1.shape[2])
X_valid_1 = X_valid_1.reshape(X_valid_1.shape[0],1,X_valid_1.shape[1],X_valid_1.shape[2])
X_test_1 = X_test_1.reshape(X_test_1.shape[0],1,X_test_1.shape[1],X_test_1.shape[2])

In [312]:
train_set_1 = Dataset(X_train_1,y_train_1)
val_set_1 = Dataset(X_valid_1,y_valid_1)
test_set_1 = Dataset(X_test_1, y_test_1)

train_loader_1 = torch.utils.data.DataLoader(train_set_1,batch_size=24,shuffle=True)
val_loader_1 = torch.utils.data.DataLoader(val_set_1,batch_size=6,shuffle=True)
test_loader_1 = torch.utils.data.DataLoader(test_set_1,batch_size=5,shuffle=True)

In [313]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()  # initial the model
        self.conv11 = nn.Conv2d(1,15,(1,11)) # depth 20, 22*990
        self.bn11 = nn.BatchNorm2d(15)
        # permute 990,  22 x 12
        self.conv12 = nn.Conv2d(990,990,(22,1)) # after permutes, 1, 15*990
        self.bn12 = nn.BatchNorm2d(990) 
        
        self.pool1 = nn.MaxPool2d((1,3),stride = (1,2)) # 1, 15*494
        # now do normal conv - maxpool
        # block2
        self.conv21 = nn.Conv2d(1,50,(15,11)) # after permute 1, 50*484
        self.bn21 = nn.BatchNorm2d(50)
        self.pool2 = nn.MaxPool2d((1,3),stride = (1,3))
        # block3
        self.conv31 = nn.Conv2d(1,100,(50,12))# after permute 1, 100*150
        self.bn31 = nn.BatchNorm2d(100)
        self.pool3 = nn.MaxPool2d((1,3),stride = (1,3))
        #block4
        self.conv41 = nn.Conv2d(1,200,(100,11)) # after permute 1, 200*40
        self.bn41 = nn.BatchNorm2d(200)
        self.pool4 = nn.MaxPool2d((1,2),stride = (1,2))
        #linear 
        self.fc1 = nn.Linear(100*50,600)
        self.fc2 = nn.Linear(600,60)
        self.fc3 = nn.Linear(60,4)
        self.drop = nn.Dropout(0.8)
 
    def forward(self,x):
        x = F.relu(self.bn11(self.conv11(x)))
        x = x.permute(0,3,2,1)
        x = F.relu(self.bn12(self.conv12(x)))
        x = x.permute(0,2,3,1)
        x = self.pool1(x)
        # now do normal 2d [conv - max] * 3
        x = F.relu(self.bn21(self.conv21(x)))
        x = x.permute(0,2,1,3)
        x = self.pool2(x)
        x = F.relu(self.bn31(self.conv31(x)))
        x = x.permute(0,2,1,3)
        x = self.pool3(x)
        #x = F.relu(self.bn41(self.conv41(x)))
        #x = x.permute(0,2,1,3)
        #x = self.pool4(x)
        
        x = x.view(-1,100*50)
        x = self.drop(F.elu(self.fc1(x)))
        x = self.drop(F.elu(self.fc2(x)))
        x = self.fc3(x)
        
        return x
net = Net()
print(net)

Net(
  (conv11): Conv2d(1, 15, kernel_size=(1, 11), stride=(1, 1))
  (bn11): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv12): Conv2d(990, 990, kernel_size=(22, 1), stride=(1, 1))
  (bn12): BatchNorm2d(990, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=(1, 3), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (conv21): Conv2d(1, 50, kernel_size=(15, 11), stride=(1, 1))
  (bn21): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (conv31): Conv2d(1, 100, kernel_size=(50, 12), stride=(1, 1))
  (bn31): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (conv41): Conv2d(1, 200, kernel_size=(100, 11), stride=(1, 1))
  (bn41): BatchNorm2d(200,

In [314]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.0005)
writer = SummaryWriter('./tensorboard/run1',flush_secs = 30) # for board 

In [316]:
val_acc_history = []
train_acc_history = []
for epoch in range(35):
    for i , data in enumerate(train_loader_1, 0):
        inputs, labels = data
        
        net.train()
        
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _,predicted = torch.max(outputs.data, 1)
        train_acc = (predicted == labels).sum().item() / len(labels)
        
        if i % 2 == 1:
            net.eval()
            val_correct, val_total = 0, 0 
            for val_data in val_loader_1:
                val_images, val_labels = val_data
                val_outputs = net(val_images)
                _, val_predicted = torch.max(val_outputs.data, 1)
                val_total += val_labels.size(0)
                val_correct += (val_predicted == val_labels).sum().item()
            val_acc = val_correct / val_total
            print('Epoch {} | Iter{} | Loss{:.4f} | TrainAcc{:.4f} | val acc {:.4f}'.format(
                epoch, i , loss, train_acc, val_acc))
            #writer.add_scalar('Train/Loss',loss,epoch*len(trainloader) + i)
            #writer.add_scalar('Train/ACC',train_acc,epoch*len(trainloader) + i)
            #writer.add_scalar('VAL/ACC',val_acc,epoch*len(trainloader) + i)
    net.eval()
    train_correct, train_total = 0, 0
    for train_data in train_loader_1:
        train_inputs, train_labels = train_data
        train_outputs = net(train_inputs)
        _, train_predicted = torch.max(train_outputs.data, 1)
        train_total += train_labels.size(0)
        train_correct += (train_predicted == train_labels).sum().item()
    train_acc = train_correct / train_total
    train_acc_history.append(train_acc)
    val_correct, val_total = 0, 0
    for val_data in val_loader_1:
        val_images, val_labels = val_data
        val_outputs = net(val_images)
        _, val_predicted = torch.max(val_outputs.data, 1)
        val_total += val_labels.size(0)
        val_correct += (val_predicted == val_labels).sum().item()
    val_acc = val_correct / val_total
    val_acc_history.append(val_acc)
    #if val_acc == max(val_acc_history):
    #    net_best = Net()
    #    net_best.load_state_dict(net.state_dict())
        
test_correct, test_total = 0, 0
for test_data in test_loader_1:
    test_images, test_labels = test_data
    test_outputs = net(test_images)
    #test_outputs = net_best(test_images)
    _, test_predicted = torch.max(test_outputs.data, 1)
    test_total += test_labels.size(0)
    test_correct += (test_predicted == test_labels).sum().item()
test_acc = test_correct / test_total
print('Test accuracy is: ',test_acc)
plt.plot(train_acc_history,)
plt.plot(val_acc_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(['train_acc','val_acc'])

Epoch 0 | Iter1 | Loss1.8423 | TrainAcc0.2083 | val acc 0.3125
Epoch 0 | Iter3 | Loss1.3950 | TrainAcc0.3333 | val acc 0.3542
Epoch 0 | Iter5 | Loss1.6417 | TrainAcc0.2917 | val acc 0.3333
Epoch 0 | Iter7 | Loss1.6836 | TrainAcc0.3333 | val acc 0.3333
Epoch 1 | Iter1 | Loss1.6417 | TrainAcc0.4167 | val acc 0.2708
Epoch 1 | Iter3 | Loss1.6027 | TrainAcc0.2500 | val acc 0.2708
Epoch 1 | Iter5 | Loss1.2947 | TrainAcc0.4167 | val acc 0.2917
Epoch 1 | Iter7 | Loss1.2956 | TrainAcc0.2381 | val acc 0.2708
Epoch 2 | Iter1 | Loss1.3836 | TrainAcc0.3333 | val acc 0.2708
Epoch 2 | Iter3 | Loss1.8769 | TrainAcc0.2500 | val acc 0.2708
Epoch 2 | Iter5 | Loss1.6363 | TrainAcc0.3333 | val acc 0.2708
Epoch 2 | Iter7 | Loss1.8593 | TrainAcc0.3333 | val acc 0.2500
Epoch 3 | Iter1 | Loss1.5921 | TrainAcc0.4167 | val acc 0.2917
Epoch 3 | Iter3 | Loss1.5385 | TrainAcc0.4167 | val acc 0.2708
Epoch 3 | Iter5 | Loss1.6260 | TrainAcc0.3750 | val acc 0.2708
Epoch 3 | Iter7 | Loss1.0672 | TrainAcc0.5714 | val acc

Epoch 32 | Iter3 | Loss0.1192 | TrainAcc0.9583 | val acc 0.2292
Epoch 32 | Iter5 | Loss0.1252 | TrainAcc0.9583 | val acc 0.2292
Epoch 32 | Iter7 | Loss0.2326 | TrainAcc0.9048 | val acc 0.2292
Epoch 33 | Iter1 | Loss0.0522 | TrainAcc0.9583 | val acc 0.2083
Epoch 33 | Iter3 | Loss0.1745 | TrainAcc0.9167 | val acc 0.2292
Epoch 33 | Iter5 | Loss0.1048 | TrainAcc0.9583 | val acc 0.2500
Epoch 33 | Iter7 | Loss0.0438 | TrainAcc1.0000 | val acc 0.2292
Epoch 34 | Iter1 | Loss0.0582 | TrainAcc1.0000 | val acc 0.2708
Epoch 34 | Iter3 | Loss0.1461 | TrainAcc0.9583 | val acc 0.2292
Epoch 34 | Iter5 | Loss0.0725 | TrainAcc0.9583 | val acc 0.2708
Epoch 34 | Iter7 | Loss0.4692 | TrainAcc0.9524 | val acc 0.2083
Test accuracy is:  0.3


NameError: name 'plt' is not defined

In [2]:
import numpy as np


In [7]:
a = np.array([[[2,3,4],[5,6,7],[1,2,3]],[[-2,-3,-4],[-5,-6,-7],[-1,-2,-3]]])
print(a)
a.reshape(1,-1)

[[[ 2  3  4]
  [ 5  6  7]
  [ 1  2  3]]

 [[-2 -3 -4]
  [-5 -6 -7]
  [-1 -2 -3]]]


array([[ 2,  3,  4,  5,  6,  7,  1,  2,  3, -2, -3, -4, -5, -6, -7, -1,
        -2, -3]])